## Part Two: Quaternion Algebras

In this part of the tutorial we will explore quaternion algebras, maximal orders and fractional ideals in SageMath, and their use in isogeny-based cryptography.

Fix $p = 45319$. We'll work with the quaternion algebra $\mathcal{B}_{p, \infty}$, the unique quaternion algebra over $\mathbb{Q}$ (up to isomorphism) ramified exactly at $p$ and $\infty$, i.e., 

$$
    \mathcal{B}_{p, \infty} = \mathbb{Q} + i\mathbb{Q} + j \mathbb{Q} + k\mathbb{Q},
$$
where $i^2 = -1$, $j^2 = -p$ and $k = ij = -ji$.



#### First step: setting up
Let's set up this quaternion algebra using SageMath.

In [ ]:
p = 45319
Bpinf = QuaternionAlgebra(QQ, -1, -p)
i,j,k = Bpinf.gens()


print(f"{Bpinf = }\n")
print(f"{Bpinf.ramified_primes() = }\n")

print(f"{Bpinf.invariants() = }")
print(f"i^2 = {i^2}")
print(f"j^2 = {j^2}\n")

print(f"Checking that i*j equals -j*i: {i*j == -j*i = }")
print(f"Checking that i*j equals k: {i*j == k = }")

#### Alternative way to setup
The quaternion algebra can also be instantiated using its *discriminant* $D = p$. 

In [ ]:
Bpinf_alt = QuaternionAlgebra(p)
print(f"Checking that Bpinf and Bpinf_alt are equivalent: {Bpinf == Bpinf_alt = }")

#### Elements of the quaternion algebra

Let's explore the elements of $\mathcal{B}_{p, \infty}$. 

For an element $\alpha \in \mathcal{B}_{p, \infty}$ where
$$
\alpha = \alpha_1 + \alpha_2 i + \alpha_3 j + \alpha_4 k,
$$
the conjugate is given by
$$
    \bar{\alpha} = \alpha_1 - \alpha_2 i - \alpha_3  j - \alpha_4  k.
$$
The reduced norm is given by 
$$
n(\alpha) = \alpha\cdot \bar{\alpha},
$$
and the reduced trace is given by 
$$
tr(\alpha) = \alpha + \bar{\alpha},
$$

In [ ]:
alpha = Bpinf.random_element()
alpha_bar = alpha.conjugate()

print(f"{alpha = }")
# We can use the function .coefficient_tuple to extract the coefficients of alpha
print(f"{alpha.coefficient_tuple() = }")

print(f"{alpha_bar = }\n")

alpha_norm = alpha.reduced_norm()
print(f"n(alpha) = {alpha_norm}")
print(f"Checking norm formula is correct: {alpha*alpha_bar == alpha_norm = }\n")

alpha_trace = alpha.reduced_trace()
print(f"tr(alpha) = {alpha_trace}")
print(f"Checking trace formula is correct: {alpha + alpha_bar == alpha_trace = }")

#### Next step: setting up a maximal order

The Deuring correspondence tells us that supersingular $j$-invaraints $j(E)$ over $\mathbb{F}_{p^2}$ are in one-to-one correspondence with maximal orders $\mathcal{O}$ in $\mathcal{B}_{p, \infty}$ via the isomorphism

$$
    \text{End}(E) \xrightarrow{\sim} \mathcal{O} 
$$


#### A concrete example

Define the supersingular elliptic curve $E_0 : y^2 = x^3 + x$ over $\mathbb{F}_{p^2}$. Here, $j(E_0) = 1728$.

Then, the endomorphism ring of this curve is isomorphic to the maximal order 

$$
    \mathcal{O}_0 = \Big\langle 1, i, \frac{i+j}{2}, \frac{1+k}{2} \Big\rangle
$$

In [ ]:
print(f"Constructing the elliptic curve...\n")
F = GF(p^2, name="z2", modulus=x^2 + 1)
z2 = F.gen()
E0 = EllipticCurve(F, [1, 0])

print(f"{E0 = }")
print(f"j(E0) = {E0.j_invariant()}")

In [ ]:

print(f"Constructing the maximal order O0...\n")

O0_basis = [1, i, (i+j)/2, (1+k)/2]
O0 = Bpinf.quaternion_order(O0_basis)

print(f"Checking that O0 is maximal using the discriminant: {O0.discriminant() == p = }")
## Here we note that there seems to be no direct in-built function that checks if an order in maximal in the quaternion algebra.


print(f"Checking the basis of O0: {O0.basis() = }")

#### Next step: integral ideals in maximal orders

$$
I = x_1\mathbb{Z} + x_2\mathbb{Z} + x_3\mathbb{Z} + x_4\mathbb{Z},
$$
where 
$$
x_1 = \frac{1}{2} + \frac{21}{2}k, \ \ \ x_2 = \frac{1}{2}i + \frac{1}{2}j + 5k, \ \ \ x_3 = j + 4k, \ \ \ x_4 = 16k,
$$

is an integral left ideal of $\mathcal{O}_0$. 

By the Deuring correspondence, $I$ corresponds to an isogeny $\phi$ of degree $n(I) = 4$ from $E_0$ to some supersingular elliptic curve $E_1/\mathbb{F}_{p^2}$, where $\text{End}(E_1) \cong \mathcal{O}_1$ and $I$ is a $\mathcal{O}_1$-right ideal. Such an ideal is called a connecting $(\mathcal{O}_0, \mathcal{O}_1)$-ideal.

The conjugate $\bar{I}$ of the ideal $I$ will correspond to the dual isogeny $\hat{\phi}$. In this way, $\bar{I}$ is a connecting $(\mathcal{O}_1, \mathcal{O}_0)$-ideal.

In [ ]:
## We note here that there is no random fractional ideal generator but we can generate a fractional ideal as
I_basis = [1/2 + 21/2*k, 1/2*i + 1/2*j + 5*k, j + 4*k, 16*k]
I = O0.left_ideal(I_basis)

print(f"{I = }")
print(f"{I in O0}")

print(f"n(I) = {I.norm()}. This will be the degree of the corresponding isogeny.")

O1 = I.right_order()
print()
print(f"{O1 = }.\nI is a right O1-ideal.\n")

I_bar = I.conjugate()
print(f"Conjugate of I is {I_bar = }")

O0_bar = I_bar.right_order()
O1_bar = I_bar.left_order()

print(f"Checking that I_bar is a connecting (O1, O0)-ideal...")
print(f"{O0_bar == O0 = }")
print(f"{O1_bar == O1 = }")

**Remark:** In Sage Math, there is no random fractional ideal sampler.

#### Next step: equivalent ideals

Consider two integral ideals $I,J$ with left order $\mathcal{O}_0$ with bases
$$
    \langle \frac{1}{2} + \frac{1}{2}i + \frac{475384310765838629}{2}j + \frac{335298457744347701}{2}k, i + 70042926510745464j + 405341384255093165k , 243016733953393400j + 243016733953393400k, 486033467906786800k \rangle,
$$
and
$$
    \langle \frac{1}{2} + 2825814j + \frac{1880109}{2}k, \frac{1}{2}i + \frac{4381057}{2}j + 2825814k, 3130583j, 3130583k\rangle, 
$$
respectively. 

$I$ and $J$ are *equivalent* right ideals. This means that they will correspond to isogenies 

$$
\phi_I: E_0 \rightarrow E_2, \\
\phi_J: E_0 \rightarrow E_2.
$$

Indeed, they are both connecting $(\mathcal{O}_0, \mathcal{O}_2)$-ideals, where $\mathcal{O}_2 \cong \text{End}(E_2)$, and 

$$
I = J\beta,
$$

for some $\beta \in \mathcal{B}_{p, \infty}^\times$.

In [ ]:
I_basis = [1/2 + 1/2*i + 475384310765838629/2*j + 335298457744347701/2*k, i + 70042926510745464*j + 405341384255093165*k, 243016733953393400*j + 243016733953393400*k, 486033467906786800*k]
I = O0.left_ideal(I_basis)

print(f"{I = }")
print(f"n(I) = {I.norm().factor()}\n")

J_basis = [1/2 + 2825814*j + 1880109/2*k, 1/2*i + 4381057/2*j + 2825814*k, 3130583*j, 3130583*k]
J = O0.left_ideal(J_basis)
print(f"{J = }")
print(f"n(I) = {J.norm().factor()}\n")

# The is_equivalent ideal only works for right ideals so we have to take conjugates...
print(f"Checking I, J are equivalent left ideals? {I.conjugate().is_equivalent(J.conjugate()) = }")
print(f"Same left order? {I.left_order() == J.left_order()}")
print(f"Same right orders? {I.left_order() == J.left_order()}\n")


print(f"To compute beta, we first need to compute the inverse of J as 'conjugate divided by the norm':")
J_inv = J.conjugate().scale(1/J.norm())
print(f"{J_inv = }")
print(f"Checking we have the inverse: {J*J_inv == O0.unit_ideal() = }\n")
beta = J_inv*I
print(f"{beta = }")
print(f"Checking relation between I,J, beta: {I == J*beta = }")

Note that $J$ has a large prime norm, whereas $I$'s norm is *powersmooth*. This means that $\phi_I$ will be much more efficient to compute that $\phi_J$. Given $J$, finding an equivalent ideal $I$ with powersmooth norm can be done using KLPT. This would be useful to implement into Sage Math. 

#### Useful aside

A useful function, in the context of SQIsign is the intersection function, which computes $I \cap J$.

In [ ]:
I.intersection(J)

#### Quick remark: elements of orders and ideals

The functionality available for elements of quaternion algebras is also available for elements of quaternion orders and ideals.